In [16]:
def find_leaders(statements):
    leaders = [statements[0]]
    for i, stmt in enumerate(statements):
        if "GOTO" in stmt:
            target_stmt = int(stmt.split()[-1]) - 1
            if target_stmt < len(statements) and statements[target_stmt] not in leaders:
                leaders.append(statements[target_stmt])
            if i + 1 < len(statements) and statements[i + 1] not in leaders:
                leaders.append(statements[i + 1])
    return sorted(leaders, key=lambda x: int(x.split('.')[0]))

def build_basic_blocks(statements, leaders):
    basic_blocks = {}
    for i, leader in enumerate(leaders):
        block_id = "B" + str(i + 1)
        block_start = int(leader.split('.')[0]) - 1
        block_end = int(leaders[i + 1].split('.')[0]) - 1 if i + 1 < len(leaders) else len(statements)
        basic_blocks[block_id] = statements[block_start:block_end]
    return basic_blocks

def build_flow_graph(statements, basic_blocks):
    flow_graph = {}
    for block_id, block_stmts in basic_blocks.items():
        if "GOTO" in block_stmts[-1]:
            target_stmt = int(block_stmts[-1].split()[-1]) - 1
            for target_block, target_stmts in basic_blocks.items():
                if statements[target_stmt] in target_stmts:
                    if block_id not in flow_graph:
                        flow_graph[block_id] = [target_block]
                    else:
                        flow_graph[block_id].append(target_block)
            if "If" in block_stmts[-1]:
                next_block = "B" + str(int(block_id[1:]) + 1)
                if block_id not in flow_graph:
                    flow_graph[block_id] = [next_block]
                else:
                    flow_graph[block_id].append(next_block)
        elif block_id != "B4":
            next_block = "B" + str(int(block_id[1:]) + 1)
            flow_graph[block_id] = [next_block]
    return flow_graph

sample_input = [
    "1. count = 0",
    "2. Result = 0",
    "3. If count > 20 GOTO 8",
    "4. count=count + 1",
    "5. increment = 2 * count",
    "6. result = result + increment",
    "7. GOTO 3",
    "8. end"
]

leaders = find_leaders(sample_input)
print("The leader statements are:")
for leader in leaders:
    print(leader)

basic_blocks = build_basic_blocks(sample_input, leaders)
print("\nThe Basic blocks are:")
for block_id, block_stmts in basic_blocks.items():
    print(f"{block_id}: contains:", ", ".join([s.split('.')[0] for s in block_stmts]))

flow_graph = build_flow_graph(sample_input, basic_blocks)
print("\nThe Program flow graph is:")
for src, dests in flow_graph.items():
    for dest in dests:
        print(f"{src} -> {dest}")


The leader statements are:
1. count = 0
3. If count > 20 GOTO 8
4. count=count + 1
8. end

The Basic blocks are:
B1: contains: 1, 2
B2: contains: 3
B3: contains: 4, 5, 6, 7
B4: contains: 8

The Program flow graph is:
B1 -> B2
B2 -> B4
B2 -> B3
B3 -> B2
